# Credit Default Risk Prediction
# Part 2. Feature Engineering

**Table of Contents**<a id='toc0_'></a>    
- 1. [Introduction   ](#toc1_)    
- 2. [Notebook preparation   ](#toc2_)    
- 3. [Feature Engineering   ](#toc3_)    
  - 3.1. [Main Home Credit Applications   ](#toc3_1_)    
  - 3.2. [Previous Credits and Their Balance from Other Institutions ](#toc3_2_)    
  - 3.3. [Previous Applications for Home Credit Loans ](#toc3_3_)    
  - 3.4. [Balance of Previous Point of Sales and Cash Loans from Home Credit ](#toc3_4_)    
  - 3.5. [Balance of Previous Credit Cards from Home Credit ](#toc3_5_)    
  - 3.6. [Installments on Previous Loans from Home Credit ](#toc3_6_)    
  - 3.7. [Summary on Feature Engineering](#toc3_7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

Note that the links in the above table of contents do not work on GitHub or Google Colab. In Google Colab, please use its embedded table of contents.

## 1. <a id='toc1_'></a>Introduction    [&#8593;](#toc0_)

This notebook is dedicated to feature engineering for the credit default risk prediction model. The focus is on enhancing the predictive capabilities of the model through informed feature engineering. It involves refining the dataset to capture underlying relationships and patterns crucial for predicting credit default risk. To begin, the challenge of disparate currencies within the dataset is addressed by transforming absolute monetary values into ratios. This ensures consistency and comparability across different currency denominations, facilitating more accurate modeling. Drawing upon insights from the experiences of predecessors tackling similar tasks, a feature aimed at identifying the mean target behavior of comparable applicants is created. Additionally, various aggregations are applied across all data tables, with diverse-span windows employed to get meaningful insights and uncover predictive patterns. 

## 2. <a id='toc2_'></a>Notebook preparation    [&#8593;](#toc0_)

Import libraries for data wrangling.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
from polars import col as c
import polars.selectors as cs
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder, OneHotEncoder
from feature_engine.selection import DropFeatures
from helper_functions import credit_data_utils
from helper_functions.feature_utils import (
    ApplicationsFeatureCreation,
    NNFeature,
    get_drop_multicollinear,
    get_drop_by_boruta,
)
import joblib
import warnings

warnings.filterwarnings("ignore")

## 3. <a id='toc3_'></a>Feature Engineering    [&#8593;](#toc0_)

All checking on whether the engineered features should be kept is done on training dataset.

In [2]:
idx_train = pl.read_csv("idx_train.csv", has_header=False)

### 3.1. <a id='toc3_1_'></a>Main Home Credit Applications    [&#8593;](#toc0_)

Based on exploratory data analysis only columns related to target and columns selected for feature engineering are loaded from main applications table.

In [3]:
columns = [
    "SK_ID_CURR",
    "TARGET",
    "NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "AMT_GOODS_PRICE",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "REGION_POPULATION_RELATIVE",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    "DAYS_REGISTRATION",
    "DAYS_ID_PUBLISH",
    "OWN_CAR_AGE",
    "FLAG_WORK_PHONE",
    "FLAG_PHONE",
    "OCCUPATION_TYPE",
    "REGION_RATING_CLIENT_W_CITY",
    "HOUR_APPR_PROCESS_START",
    "REG_CITY_NOT_LIVE_CITY",
    "REG_CITY_NOT_WORK_CITY",
    "ORGANIZATION_TYPE",
    "EXT_SOURCE_1",
    "EXT_SOURCE_2",
    "EXT_SOURCE_3",
    "APARTMENTS_AVG",
    "YEARS_BUILD_AVG",
    "FLOORSMAX_AVG",
    "COMMONAREA_AVG",
    "DEF_60_CNT_SOCIAL_CIRCLE",
    "DAYS_LAST_PHONE_CHANGE",
    "FLAG_DOCUMENT_3",
    "AMT_INCOME_TOTAL",
    "AMT_ANNUITY",
    "AMT_CREDIT",
    "CNT_CHILDREN",
    "CNT_FAM_MEMBERS",
]
appls = (
    pl.read_csv("./data/application_train.csv")
    .filter(c.SK_ID_CURR.is_in(idx_train))
    .pipe(credit_data_utils.initial_application_cleaning)
    .select(columns)
    .pipe(credit_data_utils.reduce_memory_usage_pl)
)
appls = appls.with_columns(
    CODE_GENDER=pl.when(c.CODE_GENDER == "M")
    .then(pl.lit(0))
    .when(c.CODE_GENDER == "F")
    .then(pl.lit(1))
    .otherwise(pl.lit(None))
    .cast(pl.Float32),
    DAYS_EMPLOYED=c.DAYS_EMPLOYED.fill_null(0),
    OWN_CAR_AGE=c.OWN_CAR_AGE.fill_null(0),
)
for col in appls.columns:
    column = appls[col]
    if ("NAME_" in col) | ("_TYPE" in col) | ("WALLSMATERIAL_" in col):
        appls = appls.with_columns(column.fill_null("XNA").alias(col))
appls_join = {
    "other": appls.select(["SK_ID_CURR", "TARGET"]),
    "on": "SK_ID_CURR",
    "how": "left",
}
appls = appls.to_pandas()
X_appls = appls.drop(columns=["TARGET", "SK_ID_CURR"])
y_appls = appls["TARGET"]
del appls

Size before cleaning: 415.89 MB
Initial number of columns: 122
Size after cleaning: 329.39 MB
Number of columns left: 94
Size before memory reduction: 178.17 MB
Initial data types Counter({Int64: 17, Float64: 12, String: 8, Enum(categories=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']): 1})
Size after memory reduction: 29.30 MB
Final data types Counter({Float32: 12, Categorical(ordering='physical'): 8, Int8: 7, Int32: 6, Int16: 4, Enum(categories=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']): 1})


Preprocessing contains imputation of missing values and encoding of categorical features. Target (label) encoding is selected for that. Imputation of missing values in numerical columns is important to prevent missing value propagation to engineered features. However, this all need to be set in a pipeline to prevent data leakage.

In [4]:
appls_preprocess_pipe = ColumnTransformer(
    [
        (
            "int_imp",
            SimpleImputer(strategy="most_frequent"),
            make_column_selector(pattern="FLAG_|CODE_|REG_|REGION_RATING_"),
        ),
        (
            "cont_imp",
            SimpleImputer(strategy="median"),
            make_column_selector(pattern="DAYS_|AMT_|CNT_|_AVG|EXT_|_RELATIV"),
        ),
        (
            "target_enc",
            TargetEncoder(target_type="binary", random_state=42),
            make_column_selector(dtype_include="category"),
        ),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

Custom transformers are created for feature engineering (see `feature_utils.py`in `helper_functions`). It is not possible to engineer features inplace as missing value imputation should be performed in a pipeline, which is only fitted on training data. A special feature is mean of target for 500 closest neighbors defined by the EXT_SOURCE and CREDIT_ANNUITY_RATIO features as described in this [notebook](https://www.kaggle.com/c/home-credit-default-risk/discussion/64821). The final pipeline for processing applications table includes preprocessing and feature engineering.

In [5]:
appls_full_pipe = Pipeline(
    steps=[
        ("preprocessing", appls_preprocess_pipe),
        ("feature_engineering", ApplicationsFeatureCreation()),
        ("nn_feature", NNFeature()),
    ]
).set_output(transform="pandas")

Multicollinear features are dropped using smart correlation selector, which selects features to drop based on their individual performance with logistic regression model.

In [6]:
X_appls_feat = appls_full_pipe.fit_transform(X_appls, y_appls)
drop_corr_appls = get_drop_multicollinear(
    X_appls_feat, y_appls, "drop_corr_appls", refit=False
)
X_appls_feat = X_appls_feat.drop(columns=drop_corr_appls)

Number of quasi-constant and multicollinear aggregated features to drop: 7


Boruta with random forest estimators is used to drop unimportant features.

In [7]:
drop_boruta_appls = get_drop_by_boruta(
    X_appls_feat, y_appls, "drop_boruta_appls", refit=False
)
X_appls_feat = X_appls_feat.drop(columns=drop_boruta_appls)
appls_col_num = X_appls_feat.shape[1]
print(f"Number of selected features: {appls_col_num}")

Number of unimportant aggregated features to drop using Boruta: 9
Number of selected features: 38


Based on the results, the pipeline can be updated.

In [8]:
appls_full_pipe = Pipeline(
    steps=[
        ("preprocessing", appls_preprocess_pipe),
        ("feature_engineering", ApplicationsFeatureCreation()),
        ("nn_feature", NNFeature()),
        ("drop_corr", DropFeatures(drop_corr_appls + drop_boruta_appls)),
    ]
).set_output(transform="pandas")
joblib.dump(appls_full_pipe, "./job_files/appls_full_pipe.joblib")

['./job_files/appls_full_pipe.joblib']

The final number of features from the main table to be used in modeling is 38.

In [9]:
del X_appls, y_appls, appls_preprocess_pipe

### 3.2. <a id='toc3_2_'></a>Previous Credits and Their Balance from Other Institutions  [&#8593;](#toc0_)

Status from Bureau balance table is encoded with order. Also, weighted status feature is created to put more importance on more recent status.

In [10]:
bureau_balance = pl.read_csv("./data/bureau_balance.csv").pipe(
    credit_data_utils.reduce_memory_usage_pl
)
bureau_balance = bureau_balance.with_columns(
    STATUS=pl.when(c.STATUS == "C")
    .then(pl.lit(0))
    .when(c.STATUS == "0")
    .then(pl.lit(1))
    .when(c.STATUS == "X")
    .then(pl.lit(2))
    .when(c.STATUS == "1")
    .then(pl.lit(3))
    .when(c.STATUS == "2")
    .then(pl.lit(4))
    .when(c.STATUS == "3")
    .then(pl.lit(5))
    .when(c.STATUS == "4")
    .then(pl.lit(6))
    .when(c.STATUS == "5")
    .then(pl.lit(7))
)
bureau_balance = bureau_balance.with_columns(
    WEIGHTED_STATUS=c.STATUS / (-c.MONTHS_BALANCE + 1)
)
bureau_balance.head()

Size before memory reduction: 833.13 MB
Initial data types Counter({Int64: 2, String: 1})
Size after memory reduction: 260.35 MB
Final data types Counter({Int32: 1, Int16: 1, Categorical(ordering='physical'): 1})


SK_ID_BUREAU,MONTHS_BALANCE,STATUS,WEIGHTED_STATUS
i32,i16,i32,f64
5715448,0,0,0.0
5715448,-1,0,0.0
5715448,-2,0,0.0
5715448,-3,0,0.0
5715448,-4,0,0.0


Various aggregations related to the last, average or median status are created.

In [11]:
bureau_balance_agg = bureau_balance.group_by("SK_ID_BUREAU").agg(
    MONTHS_BALANCE_LAST=c.MONTHS_BALANCE.max(),
    MONTHS_BALANCE_FIRST=c.MONTHS_BALANCE.min(),
    MONTHS_BALANCE_DURATION=c.MONTHS_BALANCE.max() - c.MONTHS_BALANCE.min(),
    MONTHS_BALANCE_MEDIAN=c.MONTHS_BALANCE.median(),
    STATUS_LAST=c.STATUS.filter(
        c.MONTHS_BALANCE == c.MONTHS_BALANCE.max()
    ).first(),
    STATUS_AVG=c.STATUS.mean(),
    STATUS_MEDIAN=c.STATUS.median(),
    STATUS_MAX=c.STATUS.max(),
    WEIGHTED_STATUS_LAST=c.WEIGHTED_STATUS.filter(
        c.MONTHS_BALANCE == c.MONTHS_BALANCE.max()
    ).first(),
    WEIGHTED_STATUS_AVG=c.WEIGHTED_STATUS.mean(),
    WEIGHTED_STATUS_MAX=c.WEIGHTED_STATUS.max(),
)
bureau_balance_agg.head()

SK_ID_BUREAU,MONTHS_BALANCE_LAST,MONTHS_BALANCE_FIRST,MONTHS_BALANCE_DURATION,MONTHS_BALANCE_MEDIAN,STATUS_LAST,STATUS_AVG,STATUS_MEDIAN,STATUS_MAX,WEIGHTED_STATUS_LAST,WEIGHTED_STATUS_AVG,WEIGHTED_STATUS_MAX
i32,i16,i16,i16,f64,i32,f64,f64,i32,f64,f64,f64
6830985,-5,-23,18,-14.0,0,0.684211,1.0,2,0.0,0.03584,0.086957
6831009,0,-48,48,-24.0,0,0.102041,0.0,1,0.0,0.002173,0.022222
5003854,0,-42,42,-21.0,1,1.0,1.0,1,1.0,0.101163,1.0
5205339,0,-4,4,-2.0,1,1.0,1.0,1,1.0,0.456667,1.0
5029197,0,-66,66,-33.0,0,0.104478,0.0,2,0.0,0.001612,0.029851


Aggregated Bureau balance table is joined to Bureau table for further feature engineering. Some ratio columns are created initially.

In [12]:
bureau = (
    pl.read_csv("./data/bureau.csv")
    .filter(c.SK_ID_CURR.is_in(idx_train))
    .pipe(credit_data_utils.initial_bureau_cleaning)
    .pipe(credit_data_utils.reduce_memory_usage_pl)
    .join(other=bureau_balance_agg, on="SK_ID_BUREAU", how="left")
)
bureau = bureau.with_columns(
    DEBT_CREDIT_RATIO=c.AMT_CREDIT_SUM_DEBT / (c.AMT_CREDIT_SUM + 1),
    OVERDUE_CREDIT_RATIO=c.AMT_CREDIT_SUM_OVERDUE / (c.AMT_CREDIT_SUM + 1),
    MAX_OVERDUE_CREDIT_RATIO=c.AMT_CREDIT_MAX_OVERDUE / (c.AMT_CREDIT_SUM + 1),
    LIMIT_CREDIT_RATIO=c.AMT_CREDIT_SUM_LIMIT / (c.AMT_CREDIT_SUM + 1),
)
bureau.head()

Size before memory reduction: 191.91 MB
Initial data types Counter({Int64: 8, Float64: 6, String: 3})
Size after memory reduction: 64.41 MB
Final data types Counter({Float32: 6, Int16: 4, Int32: 3, Categorical(ordering='physical'): 3, Int8: 1})


SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE_LAST,MONTHS_BALANCE_FIRST,MONTHS_BALANCE_DURATION,MONTHS_BALANCE_MEDIAN,STATUS_LAST,STATUS_AVG,STATUS_MEDIAN,STATUS_MAX,WEIGHTED_STATUS_LAST,WEIGHTED_STATUS_AVG,WEIGHTED_STATUS_MAX,DEBT_CREDIT_RATIO,OVERDUE_CREDIT_RATIO,MAX_OVERDUE_CREDIT_RATIO,LIMIT_CREDIT_RATIO
i32,i32,cat,cat,i16,i16,i32,i16,f32,i8,f32,f32,f32,f32,cat,i16,f32,i16,i16,i16,f64,i32,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64
215354,5714462,"""Closed""","""currency 1""",-497,0,-153,-153,0.0,0,91323.0,0.0,null,0.0,"""Consumer credi…",-131,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,null
215354,5714463,"""Active""","""currency 1""",-208,0,1075,null,0.0,0,225000.0,171342.0,null,0.0,"""Credit card""",-20,null,null,null,null,null,null,null,null,null,null,null,null,0.761517,0.0,0.0,null
215354,5714464,"""Active""","""currency 1""",-203,0,528,null,0.0,0,464323.5,null,null,0.0,"""Consumer credi…",-16,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,null
215354,5714465,"""Active""","""currency 1""",-203,0,null,null,0.0,0,90000.0,null,null,0.0,"""Credit card""",-16,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,null
215354,5714466,"""Active""","""currency 1""",-629,0,1197,null,77674.5,0,2.7e6,null,null,0.0,"""Consumer credi…",-21,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.028768,null


A dedicated function for Bureau data aggregation is created to enable aggregation on groups filtered by specific categorical columns.

In [13]:
def bureau_aggregations_func(
    df: pl.DataFrame, suffix: str = "", left_df: pl.DataFrame = None
) -> pl.DataFrame:
    """Function to aggregate bureau data and optionally add suffix to new
    columns or join with other dataframe."""
    agg_df = df.group_by("SK_ID_CURR").agg(
        cs.contains("DAYS_").mean().round().name.suffix("_AVG"),
        cs.contains("DAYS_").median().name.suffix("_MEDIAN"),
        cs.contains("DAYS_").max().name.suffix("_LAST"),
        cs.contains(["STATUS_", "MONTHS_", "_RATIO"])
        .max()
        .name.suffix("_MAX"),
        cs.contains(["STATUS_", "MONTHS_", "_RATIO"])
        .min()
        .name.suffix("_MIN"),
        cs.contains(["STATUS_", "MONTHS_", "_RATIO"])
        .mean()
        .name.suffix("_AVG"),
        CNT_CREDIT=c.SK_ID_CURR.count(),
        CNT_PROLONG_TOTAL=c.CNT_CREDIT_PROLONG.sum(),
        CREDIT_DAY_OVERDUE_TOTAL=c.CREDIT_DAY_OVERDUE.sum(),
        CREDIT_DAY_OVERDUE_MAX=c.CREDIT_DAY_OVERDUE.max(),
        DEBT_CREDIT_SUM_RATIO=(c.AMT_CREDIT_SUM_DEBT.sum())
        / (c.AMT_CREDIT_SUM.sum() + 1),
        OVERDUE_CREDIT_SUM_RATIO=(c.AMT_CREDIT_SUM_OVERDUE.sum())
        / (c.AMT_CREDIT_SUM.sum() + 1),
        MAX_OVERDUE_CREDIT_SUM_RATIO=(c.AMT_CREDIT_MAX_OVERDUE.sum())
        / (c.AMT_CREDIT_SUM.sum() + 1),
        LIMIT_CREDIT_SUM_RATIO=(c.AMT_CREDIT_SUM_LIMIT.sum())
        / (c.AMT_CREDIT_SUM.sum() + 1),
    )
    agg_df = agg_df.rename(
        lambda col: col + suffix if col != "SK_ID_CURR" else col
    )
    if left_df is not None:
        agg_df = left_df.join(other=agg_df, on="SK_ID_CURR", how="left")
    return agg_df

Finally, the function is applied and all created features are joined based on application ID.

In [14]:
bureau_agg = bureau.group_by("SK_ID_CURR").agg(
    CNT_CREDIT_TYPES_ALL=c.CREDIT_TYPE.n_unique(),
    CNT_CREDIT_TYPES_ACTIVE=c.CREDIT_TYPE.filter(
        c.CREDIT_ACTIVE == "Active"
    ).n_unique(),
    CNT_CREDIT_TYPES_CLOSED=c.CREDIT_TYPE.filter(
        c.CREDIT_ACTIVE == "Closed"
    ).n_unique(),
    ACTIVE_CREDIT_RATIO_ALL=(c.DAYS_CREDIT_ENDDATE > 0).mean(),
    ACTIVE_CREDIT_RATIO_CARD=(c.DAYS_CREDIT_ENDDATE > 0)
    .filter(c.CREDIT_TYPE == "Credit card")
    .mean(),
    ACTIVE_CREDIT_RATIO_CONSUMER=(c.DAYS_CREDIT_ENDDATE > 0)
    .filter(c.CREDIT_TYPE == "Consumer credit")
    .mean(),
)
bureau_agg = bureau_aggregations_func(bureau, "_ALL", bureau_agg)
bureau_agg = bureau.filter(c.CREDIT_ACTIVE == "Active").pipe(
    bureau_aggregations_func, "_ACTIVE", bureau_agg
)
bureau_agg = bureau.filter(c.CREDIT_ACTIVE == "Closed").pipe(
    bureau_aggregations_func, "_CLOSED", bureau_agg
)
bureau_agg = bureau.filter(c.CREDIT_TYPE == "Credit card").pipe(
    bureau_aggregations_func, "_CARD", bureau_agg
)
bureau_agg = bureau.filter(c.CREDIT_TYPE == "Consumer credit").pipe(
    bureau_aggregations_func, "_CONSUMER", bureau_agg
)
bureau_agg = bureau.filter(
    (c.CREDIT_ACTIVE == "Active") & (c.CREDIT_TYPE == "Credit card")
).pipe(bureau_aggregations_func, "_ACTIVE_CARD", bureau_agg)
bureau_agg = bureau.filter(
    (c.CREDIT_ACTIVE == "Active") & (c.CREDIT_TYPE == "Consumer credit")
).pipe(bureau_aggregations_func, "_ACTIVE_CONSUMER", bureau_agg)
bureau_agg = bureau.filter(
    (c.CREDIT_ACTIVE == "Closed") & (c.CREDIT_TYPE == "Credit card")
).pipe(bureau_aggregations_func, "_CLOSED_CARD", bureau_agg)
bureau_agg = bureau.filter(
    (c.CREDIT_ACTIVE == "Closed") & (c.CREDIT_TYPE == "Consumer credit")
).pipe(bureau_aggregations_func, "_CLOSED_CONSUMER", bureau_agg)

To enable initial removal of multicollinear and unimportant features, missing values are simply filled with 0.

In [15]:
bureau_agg_filled = (
    bureau_agg.filter(c.SK_ID_CURR.is_in(idx_train))
    .fill_null(value=0)
    .fill_nan(value=0)
    .join(**appls_join)
    .to_pandas()
)
X_bureau = bureau_agg_filled.drop(columns=["TARGET", "SK_ID_CURR"])
y_bureau = bureau_agg_filled["TARGET"]

In [16]:
drop_corr_bureau = get_drop_multicollinear(
    X_bureau, y_bureau, "drop_corr_bureau", refit=False
)
X_bureau = X_bureau.drop(columns=drop_corr_bureau)

Number of quasi-constant and multicollinear aggregated features to drop: 349


In [17]:
drop_boruta_bureau = get_drop_by_boruta(
    X_bureau, y_bureau, "drop_boruta_bureau", refit=False
)
X_bureau = X_bureau.drop(columns=drop_boruta_bureau)
bureau_col_num = X_bureau.shape[1]
print(f"Number of selected features: {bureau_col_num}")

Number of unimportant aggregated features to drop using Boruta: 133
Number of selected features: 109


Finally, 109 aggregated features for Bureau and its balance data are saved for later use. Note, that missing values are not imputed with 0 in the saved table.

In [18]:
bureau_agg.drop(drop_corr_bureau + drop_boruta_bureau).write_csv(
    "./data_agg/bureau_agg.csv"
)

In [19]:
del (
    bureau_balance,
    bureau_balance_agg,
    bureau,
    bureau_agg,
    bureau_agg_filled,
    X_bureau,
    y_bureau,
)

### 3.3. <a id='toc3_3_'></a>Previous Applications for Home Credit Loans  [&#8593;](#toc0_)

Only non-collinear columns as found during EDA is loaded and those useful for feature engineering. Only the last applications per individual contract are used for feature engineering. Missing values in categorical columns are encoded with XNA. Features describing differences and ratios of amounts are created first. A potentially very important feature related to interest of the loan is also engineered. However, it tends to be negative for revolving loans. In that case, the interest feature is set to 0.

In [20]:
columns_drop = [
    "RATE_INTEREST_PRIMARY",
    "RATE_INTEREST_PRIVILEGED",
    "DAYS_FIRST_DRAWING",
    "NAME_TYPE_SUITE",
    "WEEKDAY_APPR_PROCESS_START",
    "NAME_GOODS_CATEGORY",
    "SELLERPLACE_AREA",
    "DAYS_LAST_DUE_1ST_VERSION",
    "NFLAG_LAST_APPL_IN_DAY",
    "FLAG_LAST_APPL_PER_CONTRACT",
]
prev = (
    pl.read_csv("./data/previous_application.csv")
    .pipe(credit_data_utils.initial_prev_application_cleaning)
    .filter(c.FLAG_LAST_APPL_PER_CONTRACT == 1)
    .drop(columns_drop)
    .pipe(credit_data_utils.reduce_memory_usage_pl)
)
for col in prev.columns:
    column = prev[col]
    if (
        ("NAME_" in col)
        | ("_TYPE" in col)
        | ("CODE_" in col)
        | ("_COMBINATION" in col)
    ):
        prev = prev.with_columns(column.fill_null("XNA").alias(col))
prev = prev.with_columns(
    INTEREST=((c.CNT_PAYMENT * c.AMT_ANNUITY) - c.AMT_CREDIT).clip(
        lower_bound=0
    )  # credit card interest ends up being 0
)
prev = prev.with_columns(
    INTEREST_RATE=c.INTEREST / (c.AMT_CREDIT * c.CNT_PAYMENT + 1),
    INTEREST_SHARE=(c.INTEREST / (c.AMT_CREDIT + 1)).clip(upper_bound=1),
    CREDIT_ANNUITY_RATIO=c.AMT_CREDIT / (c.AMT_ANNUITY + 1),
    ANNUITY_CREDIT_RATIO=c.AMT_ANNUITY / (c.AMT_CREDIT + 1),
    GOODS_CREDIT_RATIO=c.AMT_GOODS_PRICE / (c.AMT_CREDIT + 1),
    GOODS_CREDIT_DIFF=c.AMT_GOODS_PRICE - c.AMT_CREDIT,
    GOODS_APPLICATION_RATIO=c.AMT_GOODS_PRICE / (c.AMT_APPLICATION + 1),
    GOODS_APPLICATION_DIFF=c.AMT_GOODS_PRICE - c.AMT_APPLICATION,
    APPLICATION_CREDIT_RATIO=c.AMT_APPLICATION / (c.AMT_CREDIT + 1),
    APPLICATION_CREDIT_DIFF=c.AMT_APPLICATION - c.AMT_CREDIT,
)
prev.head()

Size before cleaning: 776.58 MB
Size after cleaning: 759.95 MB
Number of columns left: 37
Size before memory reduction: 578.87 MB
Initial data types Counter({String: 12, Int64: 8, Float64: 6, Enum(categories=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']): 1})
Size after memory reduction: 154.35 MB
Final data types Counter({Categorical(ordering='physical'): 12, Float32: 6, Int16: 5, Int32: 2, Enum(categories=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']): 1, Int8: 1})


SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DUE,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,INTEREST,INTEREST_RATE,INTEREST_SHARE,CREDIT_ANNUITY_RATIO,ANNUITY_CREDIT_RATIO,GOODS_CREDIT_RATIO,GOODS_CREDIT_DIFF,GOODS_APPLICATION_RATIO,GOODS_APPLICATION_DIFF,APPLICATION_CREDIT_RATIO,APPLICATION_CREDIT_DIFF
i32,i32,cat,f32,f32,f32,f32,f32,enum,f32,cat,cat,i16,cat,cat,cat,cat,cat,cat,cat,i16,cat,cat,i16,i16,i16,i8,f32,f64,f64,f64,f64,f64,f32,f64,f32,f64,f32
2030495,271877,"""Consumer loans…",1730.430054,17145.0,17145.0,0.0,17145.0,"""15""",0.0,"""XAP""","""Approved""",-73,"""Cash through t…","""XAP""","""Repeater""","""POS""","""XNA""","""Country-wide""","""Connectivity""",12,"""middle""","""POS mobile wit…",-42,-42,-37,0,3620.160156,0.017596,0.211137,9.902219,0.100923,0.999942,0.0,0.999942,0.0,0.999942,0.0
2802425,108129,"""Cash loans""",25188.615234,607500.0,679671.0,null,607500.0,"""11""",null,"""XNA""","""Approved""",-164,"""XNA""","""XAP""","""Repeater""","""Cash""","""x-sell""","""Contact center…","""XNA""",36,"""low_action""","""Cash X-Sell""",-134,null,null,1,227119.125,0.009282,0.33416,26.982191,0.03706,0.893813,-72171.0,0.999998,0.0,0.893813,-72171.0
2523466,122040,"""Cash loans""",15060.735352,112500.0,136444.5,null,112500.0,"""11""",null,"""XNA""","""Approved""",-301,"""Cash through t…","""XAP""","""Repeater""","""Cash""","""x-sell""","""Credit and cas…","""XNA""",12,"""high""","""Cash X-Sell""",-271,null,null,1,44284.328125,0.027047,0.324557,9.059016,0.110379,0.824505,-23944.5,0.999991,0.0,0.824505,-23944.5
2819243,176158,"""Cash loans""",47041.335938,450000.0,470790.0,null,450000.0,"""7""",null,"""XNA""","""Approved""",-512,"""Cash through t…","""XAP""","""Repeater""","""Cash""","""x-sell""","""Credit and cas…","""XNA""",12,"""middle""","""Cash X-Sell""",-482,-182,-177,1,93706.0,0.016587,0.199039,10.007794,0.09992,0.955838,-20790.0,0.999998,0.0,0.955838,-20790.0
1784265,202054,"""Cash loans""",31924.394531,337500.0,404055.0,null,337500.0,"""9""",null,"""Repairs""","""Refused""",-781,"""Cash through t…","""HC""","""Repeater""","""Cash""","""walk-in""","""Credit and cas…","""XNA""",24,"""high""","""Cash Street""",null,null,null,null,362130.5,0.037343,0.896238,12.656226,0.07901,0.83528,-66555.0,0.999997,0.0,0.83528,-66555.0


To enable aggregations of categorical features they are one-hot-encoded.

In [21]:
def custom_combiner(feature, category):
    if str(category) == "infrequent_sklearn":
        name = "OHE_" + str(feature).split("_", 1)[-1] + "_RARE"
    else:
        name = (
            "OHE_"
            + str(feature).split("_", 1)[-1]
            + "_"
            + str(category)
            .upper()
            .replace("/ ", "")
            .replace(" ", "_")
            .replace("-", "_")
        )
    return name


cat_columns = [
    col
    for col in prev.columns
    if any(sub in col for sub in ["NAME_", "PRODUCT_", "CODE_", "_TYPE"])
]
ohe_pipe = ColumnTransformer(
    [
        (
            "ohe",
            OneHotEncoder(
                min_frequency=0.05,
                sparse_output=False,
                dtype=np.int8,
                feature_name_combiner=custom_combiner,
            ),
            cat_columns,
        ),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="polars")
ohe_pipe.fit(prev.filter(c.SK_ID_CURR.is_in(idx_train)))
prev = prev.select(["SK_ID_PREV"] + cat_columns).join(
    other=ohe_pipe.transform(prev),
    on="SK_ID_PREV",
    how="left",
)
prev.head()

SK_ID_PREV,NAME_CONTRACT_TYPE,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION,OHE_CONTRACT_TYPE_CASH_LOANS,OHE_CONTRACT_TYPE_CONSUMER_LOANS,OHE_CONTRACT_TYPE_REVOLVING_LOANS,OHE_CONTRACT_TYPE_RARE,OHE_CASH_LOAN_PURPOSE_XAP,OHE_CASH_LOAN_PURPOSE_XNA,OHE_CASH_LOAN_PURPOSE_RARE,OHE_CONTRACT_STATUS_APPROVED,OHE_CONTRACT_STATUS_CANCELED,OHE_CONTRACT_STATUS_REFUSED,OHE_CONTRACT_STATUS_RARE,OHE_PAYMENT_TYPE_CASH_THROUGH_THE_BANK,OHE_PAYMENT_TYPE_XNA,OHE_PAYMENT_TYPE_RARE,OHE_REJECT_REASON_HC,OHE_REJECT_REASON_XAP,OHE_REJECT_REASON_RARE,OHE_CLIENT_TYPE_NEW,OHE_CLIENT_TYPE_REFRESHED,OHE_CLIENT_TYPE_REPEATER,OHE_CLIENT_TYPE_RARE,OHE_PORTFOLIO_CARDS,OHE_PORTFOLIO_CASH,OHE_PORTFOLIO_POS,…,OHE_YIELD_GROUP_LOW_ACTION,OHE_YIELD_GROUP_LOW_NORMAL,OHE_YIELD_GROUP_MIDDLE,OHE_COMBINATION_CARD_STREET,OHE_COMBINATION_CASH,OHE_COMBINATION_CASH_STREET,OHE_COMBINATION_CASH_X_SELL,OHE_COMBINATION_POS_HOUSEHOLD_WITH_INTEREST,OHE_COMBINATION_POS_HOUSEHOLD_WITHOUT_INTEREST,OHE_COMBINATION_POS_INDUSTRY_WITH_INTEREST,OHE_COMBINATION_POS_MOBILE_WITH_INTEREST,OHE_COMBINATION_RARE,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DUE,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,INTEREST,INTEREST_RATE,INTEREST_SHARE,CREDIT_ANNUITY_RATIO,ANNUITY_CREDIT_RATIO,GOODS_CREDIT_RATIO,GOODS_CREDIT_DIFF,GOODS_APPLICATION_RATIO,GOODS_APPLICATION_DIFF,APPLICATION_CREDIT_RATIO,APPLICATION_CREDIT_DIFF
i32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i32,f32,f32,f32,f32,f32,enum,f32,i16,i16,i16,i16,i16,i8,f32,f64,f64,f64,f64,f64,f32,f64,f32,f64,f32
2030495,"""Consumer loans…","""XAP""","""Approved""","""Cash through t…","""XAP""","""Repeater""","""POS""","""XNA""","""Country-wide""","""Connectivity""","""middle""","""POS mobile wit…",0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,…,0,0,1,0,0,0,0,0,0,0,1,0,271877,1730.430054,17145.0,17145.0,0.0,17145.0,"""15""",0.0,-73,12,-42,-42,-37,0,3620.160156,0.017596,0.211137,9.902219,0.100923,0.999942,0.0,0.999942,0.0,0.999942,0.0
2802425,"""Cash loans""","""XNA""","""Approved""","""XNA""","""XAP""","""Repeater""","""Cash""","""x-sell""","""Contact center…","""XNA""","""low_action""","""Cash X-Sell""",1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,…,1,0,0,0,0,0,1,0,0,0,0,0,108129,25188.615234,607500.0,679671.0,null,607500.0,"""11""",null,-164,36,-134,null,null,1,227119.125,0.009282,0.33416,26.982191,0.03706,0.893813,-72171.0,0.999998,0.0,0.893813,-72171.0
2523466,"""Cash loans""","""XNA""","""Approved""","""Cash through t…","""XAP""","""Repeater""","""Cash""","""x-sell""","""Credit and cas…","""XNA""","""high""","""Cash X-Sell""",1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,…,0,0,0,0,0,0,1,0,0,0,0,0,122040,15060.735352,112500.0,136444.5,null,112500.0,"""11""",null,-301,12,-271,null,null,1,44284.328125,0.027047,0.324557,9.059016,0.110379,0.824505,-23944.5,0.999991,0.0,0.824505,-23944.5
2819243,"""Cash loans""","""XNA""","""Approved""","""Cash through t…","""XAP""","""Repeater""","""Cash""","""x-sell""","""Credit and cas…","""XNA""","""middle""","""Cash X-Sell""",1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,…,0,0,1,0,0,0,1,0,0,0,0,0,176158,47041.335938,450000.0,470790.0,null,450000.0,"""7""",null,-512,12,-482,-182,-177,1,93706.0,0.016587,0.199039,10.007794,0.09992,0.955838,-20790.0,0.999998,0.0,0.955838,-20790.0
1784265,"""Cash loans""","""Repairs""","""Refused""","""Cash through t…","""HC""","""Repeater""","""Cash""","""walk-in""","""Credit and cas…","""XNA""","""high""","""Cash Street""",1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,…,0,0,0,0,0,1,0,0,0,0,0,0,202054,31924.394531,337500.0,404055.0,null,337500.0,"""9""",null,-781,24,null,null,nul

Again, function is written to enable aggregations over data grouped by different columns.

In [22]:
def previous_aggregations_func(
    df: pl.DataFrame, suffix: str = "", left_df: pl.DataFrame = None
) -> pl.DataFrame:
    """Function to aggregate previous applications data and optionally add
    suffix to new columns or join with other dataframe."""
    agg_df = df.group_by("SK_ID_CURR").agg(
        cs.contains("OHE_").sum().name.suffix("_CNT"),
        cs.contains(["DAYS_", "CNT_"]).mean().round().name.suffix("_AVG"),
        cs.contains(["DAYS_", "CNT_"]).median().name.suffix("_MEDIAN"),
        cs.contains(["DAYS_", "CNT_"]).max().name.suffix("_MAX"),
        cs.contains(["AMT_", "RATE_", "_RATIO", "_DIFF", "INTEREST_"])
        .max()
        .name.suffix("_MAX"),
        cs.contains(["AMT_", "RATE_", "_RATIO", "_DIFF", "INTEREST_"])
        .min()
        .name.suffix("_MIN"),
        cs.contains(["AMT_", "RATE_", "_RATIO", "_DIFF", "INTEREST_"])
        .mean()
        .name.suffix("_AVG"),
        CNT_APPLS=c.SK_ID_CURR.count(),
    )
    agg_df = agg_df.rename(
        lambda col: col + suffix if col != "SK_ID_CURR" else col
    )
    if left_df is not None:
        agg_df = left_df.join(other=agg_df, on="SK_ID_CURR", how="left")
    return agg_df

Finally aggregations over previous application data are run and joined to form a dataset of 964 features.

In [23]:
prev_agg = prev.group_by("SK_ID_CURR").agg(
    (cs.all() - cs.contains(["SK_ID_", "OHE_"]))
    .filter(c.DAYS_DECISION == c.DAYS_DECISION.max())
    .first()
    .name.suffix("_LAST")
)
prev_agg = previous_aggregations_func(prev, "_ALL", prev_agg)
prev_agg = prev.filter(c.NAME_CONTRACT_TYPE == "Consumer loans").pipe(
    previous_aggregations_func, "_CONSUMER", prev_agg
)
prev_agg = prev.filter(c.NAME_CONTRACT_TYPE == "Cash loans").pipe(
    previous_aggregations_func, "_CASH", prev_agg
)
prev_agg = prev.filter(c.NAME_CONTRACT_TYPE == "Revolving loans").pipe(
    previous_aggregations_func, "_REVOLVING", prev_agg
)
prev_agg = prev.filter(c.NAME_CONTRACT_STATUS == "Approved").pipe(
    previous_aggregations_func, "_APPROVED", prev_agg
)
prev_agg = prev.filter(c.NAME_CONTRACT_STATUS == "Refused").pipe(
    previous_aggregations_func, "_REFUSED", prev_agg
)
prev_agg = (
    prev.sort(c.DAYS_DECISION, descending=True)
    .group_by("SK_ID_CURR")
    .head(5)
    .pipe(previous_aggregations_func, "_5LAST", prev_agg)
)
prev_agg = (
    prev.sort(c.DAYS_DECISION, descending=True)
    .group_by("SK_ID_CURR")
    .head(3)
    .pipe(previous_aggregations_func, "_3LAST", prev_agg)
)
prev_agg.head()

SK_ID_CURR,NAME_CONTRACT_TYPE_LAST,NAME_CASH_LOAN_PURPOSE_LAST,NAME_CONTRACT_STATUS_LAST,NAME_PAYMENT_TYPE_LAST,CODE_REJECT_REASON_LAST,NAME_CLIENT_TYPE_LAST,NAME_PORTFOLIO_LAST,NAME_PRODUCT_TYPE_LAST,CHANNEL_TYPE_LAST,NAME_SELLER_INDUSTRY_LAST,NAME_YIELD_GROUP_LAST,PRODUCT_COMBINATION_LAST,AMT_ANNUITY_LAST,AMT_APPLICATION_LAST,AMT_CREDIT_LAST,AMT_DOWN_PAYMENT_LAST,AMT_GOODS_PRICE_LAST,HOUR_APPR_PROCESS_START_LAST,RATE_DOWN_PAYMENT_LAST,DAYS_DECISION_LAST,CNT_PAYMENT_LAST,DAYS_FIRST_DUE_LAST,DAYS_LAST_DUE_LAST,DAYS_TERMINATION_LAST,NFLAG_INSURED_ON_APPROVAL_LAST,INTEREST_LAST,INTEREST_RATE_LAST,INTEREST_SHARE_LAST,CREDIT_ANNUITY_RATIO_LAST,ANNUITY_CREDIT_RATIO_LAST,GOODS_CREDIT_RATIO_LAST,GOODS_CREDIT_DIFF_LAST,GOODS_APPLICATION_RATIO_LAST,GOODS_APPLICATION_DIFF_LAST,APPLICATION_CREDIT_RATIO_LAST,APPLICATION_CREDIT_DIFF_LAST,…,GOODS_APPLICATION_RATIO_MAX_3LAST,GOODS_APPLICATION_DIFF_MAX_3LAST,APPLICATION_CREDIT_RATIO_MAX_3LAST,APPLICATION_CREDIT_DIFF_MAX_3LAST,AMT_ANNUITY_MIN_3LAST,AMT_APPLICATION_MIN_3LAST,AMT_CREDIT_MIN_3LAST,AMT_DOWN_PAYMENT_MIN_3LAST,AMT_GOODS_PRICE_MIN_3LAST,RATE_DOWN_PAYMENT_MIN_3LAST,INTEREST_RATE_MIN_3LAST,INTEREST_SHARE_MIN_3LAST,CREDIT_ANNUITY_RATIO_MIN_3LAST,ANNUITY_CREDIT_RATIO_MIN_3LAST,GOODS_CREDIT_RATIO_MIN_3LAST,GOODS_CREDIT_DIFF_MIN_3LAST,GOODS_APPLICATION_RATIO_MIN_3LAST,GOODS_APPLICATION_DIFF_MIN_3LAST,APPLICATION_CREDIT_RATIO_MIN_3LAST,APPLICATION_CREDIT_DIFF_MIN_3LAST,AMT_ANNUITY_AVG_3LAST,AMT_APPLICATION_AVG_3LAST,AMT_CREDIT_AVG_3LAST,AMT_DOWN_PAYMENT_AVG_3LAST,AMT_GOODS_PRICE_AVG_3LAST,RATE_DOWN_PAYMENT_AVG_3LAST,INTEREST_RATE_AVG_3LAST,INTEREST_SHARE_AVG_3LAST,CREDIT_ANNUITY_RATIO_AVG_3LAST,ANNUITY_CREDIT_RATIO_AVG_3LAST,GOODS_CREDIT_RATIO_AVG_3LAST,GOODS_CREDIT_DIFF_AVG_3LAST,GOODS_APPLICATION_RATIO_AVG_3LAST,GOODS_APPLICATION_DIFF_AVG_3LAST,APPLICATION_CREDIT_RATIO_AVG_3LAST,APPLICATION_CREDIT_DIFF_AVG_3LAST,CNT_APPLS_3LAST
i32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,enum,f32,i16,i16,i16,i16,i16,i8,f32,f64,f64,f64,f64,f64,f32,f64,f32,f64,f32,…,f64,f32,f64,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f64,f64,f32,f64,f32,f64,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f64,f64,f32,f64,f32,f64,f32,u32
332209,"""Cash loans""","""XNA""","""Refused""","""Cash through t…","""HC""","""Repeater""","""Cash""","""x-sell""","""AP+ (Cash loan…","""XNA""","""high""","""Cash X-Sell""",27729.449219,360000.0,409896.0,null,360000.0,"""10""",null,-749,36,null,null,null,null,588364.1875,0.039872,1.0,14.781441,0.06765,0.878269,-49896.0,0.999997,0.0,0.878269,-49896.0,…,0.999998,0.0,0.884419,-49896.0,24487.650391,360000.0,409896.0,0.0,360000.0,0.0,0.015002,0.540079,14.781441,0.04278,0.878269,-66159.0,0.999997,0.0,0.878269,-66159.0,27226.546875,416250.0,472606.5,0.0,416250.0,0.0,0.031582,0.846693,17.645792,0.05936,0.880319,-56356.5,0.999998,0.0,0.880319,-56356.5,3
289991,"""Revolving loan…","""XAP""","""Approved""","""XNA""","""XAP""","""Refreshed""","""Cards""","""walk-in""","""Regional / Loc…","""Consumer elect…","""XNA""","""Card Street""",2250.0,45000.0,45000.0,null,45000.0,"""9""",null,-300,0,null,null,null,0,0.0,0.0,0.0,19.991115,0.049999,0.999978,0.0,0.999978,0.0,0.999978,0.0,…,0.999998,0.0,1.013225,1786.5,2250.0,45000.0,45000.0,0.0,45000.0,0.0,0.0,0.0,9.371421,0.049999,0.883387,-57501.0,0.999978,0.0,0.883387,-57501.0,14044.831055,205800.0,224371.5,6975.0,205800.0,0.050995,0.011123,0.173493,16.23845,0.069456,0.96553,-18571.5,0.999989,0.0,0.96553,-18571.5,3
111610,"""Revolving loan…","""XAP""","""Refused""","""XNA""","""HC""","""Repeater""","""Cards""","""walk-in""","""Country-wide""","""Consumer elect…","""XNA""","""Card Street""",2250.0,45000.0,45000.0,null,45000.0,"""15""",null,-33,0,null,null,null,null,0.0,0.0,0.0,19.991115,0.049999,0.999978,0.0,0.999978,0.0,0.999978,0.0,…,0.999978,0.0,0.999978,0.0,2122.514893,11695.5,11695.5,0.0,11695.5,0.0,0.0,0.0,5.507614,0.049999,0.999915,0.0,0.999915,0.0,0.999915,0.0,2207.504883,33898.5,33898.5,0.0,33898.5,0.0,0.004938,0.029627,15.163281,0.09

Since there are still categorical features in the dataset, target encoding is used to enable elimination of multicollinear and unimportant features.

In [24]:
enc_pipe = ColumnTransformer(
    [
        (
            "target_enc",
            TargetEncoder(target_type="binary", random_state=42),
            make_column_selector(dtype_include="category"),
        ),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

prev_agg_filled = (
    prev_agg.filter(c.SK_ID_CURR.is_in(idx_train))
    .fill_null(value=0)
    .fill_nan(value=0)
    .join(**appls_join)
    .to_pandas()
)
X_prev = prev_agg_filled.drop(columns=["TARGET", "SK_ID_CURR"])
y_prev = prev_agg_filled["TARGET"]
X_prev = enc_pipe.fit_transform(X_prev, y_prev)

In [25]:
drop_corr_prev = get_drop_multicollinear(
    X_prev, y_prev, "drop_corr_prev", refit=False
)
X_prev = X_prev.drop(columns=drop_corr_prev)

Number of quasi-constant and multicollinear aggregated features to drop: 519


In [26]:
drop_boruta_prev = get_drop_by_boruta(
    X_prev, y_prev, "drop_boruta_prev", refit=False
)
X_prev = X_prev.drop(columns=drop_boruta_prev)
prev_col_num = X_prev.shape[1]
print(f"Number of selected features: {prev_col_num}")

Number of unimportant aggregated features to drop using Boruta: 174
Number of selected features: 271


Finally, 271 aggregated feature for previous application data is saved for later use. Note, that missing values are not imputed with 0 and categorical features are not encoded in the saved table.

In [27]:
prev_agg.drop(drop_corr_prev + drop_boruta_prev).write_csv(
    "./data_agg/prev_agg.csv"
)

In [28]:
del (
    prev,
    prev_agg,
    prev_agg_filled,
    X_prev,
    y_prev,
)

### 3.4. <a id='toc3_4_'></a>Balance of Previous Point of Sales and Cash Loans from Home Credit  [&#8593;](#toc0_)

Before aggregation, missing values in a single categorical column from POS and cash loan balance data are filled with XNA.

In [29]:
pos_cash = (
    pl.read_csv("./data/POS_CASH_balance.csv")
    .pipe(credit_data_utils.initial_pos_cash_credit_card_installments_cleaning)
    .pipe(credit_data_utils.reduce_memory_usage_pl)
)
pos_cash = pos_cash.with_columns(c.NAME_CONTRACT_STATUS.fill_null("XNA"))
pos_cash.head()

Size before cleaning: 689.24 MB
Size after cleaning: 690.43 MB
Number of columns left: 8
Size before memory reduction: 690.43 MB
Initial data types Counter({Int64: 7, String: 1})
Size after memory reduction: 251.57 MB
Final data types Counter({Int32: 4, Int16: 3, Categorical(ordering='physical'): 1})


SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
i32,i32,i16,i16,i16,cat,i32,i32
1803195,182943,-31,48,45,"""Active""",0,0
1715348,367990,-33,36,35,"""Active""",0,0
1784872,397406,-32,12,9,"""Active""",0,0
1903291,269225,-35,48,42,"""Active""",0,0
2341044,334279,-35,36,35,"""Active""",0,0


First some aggregations are done on individual loans to have information on mean, median, sum, last, minimum or maximum values.

In [30]:
pos_cash_agg_initial = pos_cash.group_by(["SK_ID_PREV", "SK_ID_CURR"]).agg(
    cs.contains(["SK_DPD", "NAME_", "MONTHS_", "_FUTURE"])
    .filter(c.MONTHS_BALANCE == c.MONTHS_BALANCE.max())
    .first()
    .name.suffix("_LAST"),
    cs.contains("SK_DPD").max().name.suffix("_MAX"),
    cs.contains("SK_DPD").sum().name.suffix("_SUM"),
    cs.contains("SK_DPD").mean().round().name.suffix("_AVG"),
    MONTHS_BALANCE_FIRST=c.MONTHS_BALANCE.min(),
    MONTHS_BALANCE_DURATION=c.MONTHS_BALANCE.max() - c.MONTHS_BALANCE.min(),
    MONTHS_BALANCE_MEDIAN=c.MONTHS_BALANCE.median(),
    CNT_INSTALMENT_MEDIAN=c.CNT_INSTALMENT.median(),
)
pos_cash_agg_initial.head()

SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE_LAST,CNT_INSTALMENT_FUTURE_LAST,NAME_CONTRACT_STATUS_LAST,SK_DPD_LAST,SK_DPD_DEF_LAST,SK_DPD_MAX,SK_DPD_DEF_MAX,SK_DPD_SUM,SK_DPD_DEF_SUM,SK_DPD_AVG,SK_DPD_DEF_AVG,MONTHS_BALANCE_FIRST,MONTHS_BALANCE_DURATION,MONTHS_BALANCE_MEDIAN,CNT_INSTALMENT_MEDIAN
i32,i32,i16,i16,cat,i32,i32,i32,i32,i32,i32,f64,f64,i16,i16,f64,f64
1715348,367990,-16,0,"""Completed""",0,0,0,0,0,0,0.0,0.0,-34,18,-25.0,36.0
2341044,334279,-1,1,"""Active""",0,0,0,0,0,0,0.0,0.0,-36,35,-18.5,36.0
2207092,342166,-20,0,"""Completed""",0,0,0,0,0,0,0.0,0.0,-32,12,-26.0,12.0
1387235,153211,-1,2,"""Active""",0,0,0,0,0,0,0.0,0.0,-35,34,-18.0,36.0
1716688,405549,-32,0,"""Completed""",0,0,0,0,0,0,0.0,0.0,-44,12,-38.0,12.0


Again, function is written to apply aggregations over different groups of data.

In [31]:
def pos_cash_aggregations_func(
    df: pl.DataFrame, suffix: str = "", left_df: pl.DataFrame = None
) -> pl.DataFrame:
    """Function to aggregate POS and cash balance data and optionally add
    suffix to new columns or join with other dataframe."""
    agg_df = df.group_by("SK_ID_CURR").agg(
        (cs.all() - cs.contains(["SK_ID_", "NAME_"]))
        .mean()
        .round()
        .name.suffix("_AVG"),
        (cs.all() - cs.contains(["SK_ID_", "NAME_"]))
        .median()
        .name.suffix("_MEDIAN"),
        (cs.all() - cs.contains(["SK_ID_", "NAME_"]))
        .max()
        .name.suffix("_MAX"),
        (cs.all() - cs.contains(["SK_ID_", "NAME_"]))
        .min()
        .name.suffix("_MIN"),
        CNT=c.SK_ID_CURR.count(),
    )
    agg_df = agg_df.rename(
        lambda col: col + suffix if col != "SK_ID_CURR" else col
    )
    if left_df is not None:
        agg_df = left_df.join(other=agg_df, on="SK_ID_CURR", how="left")
    return agg_df

These aggregations are applied over the whole dataset and also only on the information from the last two years.

In [32]:
pos_cash_agg = pos_cash_aggregations_func(pos_cash_agg_initial, "_ALL")
pos_cash_agg = pos_cash_agg_initial.filter(
    c.NAME_CONTRACT_STATUS_LAST == "Completed"
).pipe(pos_cash_aggregations_func, "_COMPLETE", pos_cash_agg)
pos_cash_agg = pos_cash_agg_initial.filter(
    c.NAME_CONTRACT_STATUS_LAST != "Completed"
).pipe(pos_cash_aggregations_func, "_NONCOMPLETE", pos_cash_agg)
pos_cash_agg = pos_cash_agg_initial.filter(c.MONTHS_BALANCE_LAST >= -24).pipe(
    pos_cash_aggregations_func, "_ALL_2YRS", pos_cash_agg
)
pos_cash_agg = pos_cash_agg_initial.filter(
    (c.MONTHS_BALANCE_LAST >= -24)
    & (c.NAME_CONTRACT_STATUS_LAST == "Completed")
).pipe(pos_cash_aggregations_func, "_COMPLETE_2YRS", pos_cash_agg)
pos_cash_agg = pos_cash_agg_initial.filter(
    (c.MONTHS_BALANCE_LAST >= -24)
    & (c.NAME_CONTRACT_STATUS_LAST != "Completed")
).pipe(pos_cash_aggregations_func, "_NONCOMPLETE_2YRS", pos_cash_agg)
pos_cash_agg = pos_cash_agg.with_columns(
    COMPLETE_ALL_RATIO=c.CNT_COMPLETE / c.CNT_ALL,
    COMPLETE_ALL_2YRS_RATIO=c.CNT_COMPLETE_2YRS / c.CNT_ALL_2YRS,
    NONCOMPLETE_ALL_RATIO=c.CNT_NONCOMPLETE / c.CNT_ALL,
    NONCOMPLETE_ALL_2YRS_RATIO=c.CNT_NONCOMPLETE_2YRS / c.CNT_ALL_2YRS,
)
pos_cash_agg.head()

SK_ID_CURR,MONTHS_BALANCE_LAST_AVG_ALL,CNT_INSTALMENT_FUTURE_LAST_AVG_ALL,SK_DPD_LAST_AVG_ALL,SK_DPD_DEF_LAST_AVG_ALL,SK_DPD_MAX_AVG_ALL,SK_DPD_DEF_MAX_AVG_ALL,SK_DPD_SUM_AVG_ALL,SK_DPD_DEF_SUM_AVG_ALL,SK_DPD_AVG_AVG_ALL,SK_DPD_DEF_AVG_AVG_ALL,MONTHS_BALANCE_FIRST_AVG_ALL,MONTHS_BALANCE_DURATION_AVG_ALL,MONTHS_BALANCE_MEDIAN_AVG_ALL,CNT_INSTALMENT_MEDIAN_AVG_ALL,MONTHS_BALANCE_LAST_MEDIAN_ALL,CNT_INSTALMENT_FUTURE_LAST_MEDIAN_ALL,SK_DPD_LAST_MEDIAN_ALL,SK_DPD_DEF_LAST_MEDIAN_ALL,SK_DPD_MAX_MEDIAN_ALL,SK_DPD_DEF_MAX_MEDIAN_ALL,SK_DPD_SUM_MEDIAN_ALL,SK_DPD_DEF_SUM_MEDIAN_ALL,SK_DPD_AVG_MEDIAN_ALL,SK_DPD_DEF_AVG_MEDIAN_ALL,MONTHS_BALANCE_FIRST_MEDIAN_ALL,MONTHS_BALANCE_DURATION_MEDIAN_ALL,MONTHS_BALANCE_MEDIAN_MEDIAN_ALL,CNT_INSTALMENT_MEDIAN_MEDIAN_ALL,MONTHS_BALANCE_LAST_MAX_ALL,CNT_INSTALMENT_FUTURE_LAST_MAX_ALL,SK_DPD_LAST_MAX_ALL,SK_DPD_DEF_LAST_MAX_ALL,SK_DPD_MAX_MAX_ALL,SK_DPD_DEF_MAX_MAX_ALL,SK_DPD_SUM_MAX_ALL,SK_DPD_DEF_SUM_MAX_ALL,…,MONTHS_BALANCE_FIRST_MEDIAN_NONCOMPLETE_2YRS,MONTHS_BALANCE_DURATION_MEDIAN_NONCOMPLETE_2YRS,MONTHS_BALANCE_MEDIAN_MEDIAN_NONCOMPLETE_2YRS,CNT_INSTALMENT_MEDIAN_MEDIAN_NONCOMPLETE_2YRS,MONTHS_BALANCE_LAST_MAX_NONCOMPLETE_2YRS,CNT_INSTALMENT_FUTURE_LAST_MAX_NONCOMPLETE_2YRS,SK_DPD_LAST_MAX_NONCOMPLETE_2YRS,SK_DPD_DEF_LAST_MAX_NONCOMPLETE_2YRS,SK_DPD_MAX_MAX_NONCOMPLETE_2YRS,SK_DPD_DEF_MAX_MAX_NONCOMPLETE_2YRS,SK_DPD_SUM_MAX_NONCOMPLETE_2YRS,SK_DPD_DEF_SUM_MAX_NONCOMPLETE_2YRS,SK_DPD_AVG_MAX_NONCOMPLETE_2YRS,SK_DPD_DEF_AVG_MAX_NONCOMPLETE_2YRS,MONTHS_BALANCE_FIRST_MAX_NONCOMPLETE_2YRS,MONTHS_BALANCE_DURATION_MAX_NONCOMPLETE_2YRS,MONTHS_BALANCE_MEDIAN_MAX_NONCOMPLETE_2YRS,CNT_INSTALMENT_MEDIAN_MAX_NONCOMPLETE_2YRS,MONTHS_BALANCE_LAST_MIN_NONCOMPLETE_2YRS,CNT_INSTALMENT_FUTURE_LAST_MIN_NONCOMPLETE_2YRS,SK_DPD_LAST_MIN_NONCOMPLETE_2YRS,SK_DPD_DEF_LAST_MIN_NONCOMPLETE_2YRS,SK_DPD_MAX_MIN_NONCOMPLETE_2YRS,SK_DPD_DEF_MAX_MIN_NONCOMPLETE_2YRS,SK_DPD_SUM_MIN_NONCOMPLETE_2YRS,SK_DPD_DEF_SUM_MIN_NONCOMPLETE_2YRS,SK_DPD_AVG_MIN_NONCOMPLETE_2YRS,SK_DPD_DEF_AVG_MIN_NONCOMPLETE_2YRS,MONTHS_BALANCE_FIRST_MIN_NONCOMPLETE_2YRS,MONTHS_BALANCE_DURATION_MIN_NONCOMPLETE_2YRS,MONTHS_BALANCE_MEDIAN_MIN_NONCOMPLETE_2YRS,CNT_INSTALMENT_MEDIAN_MIN_NONCOMPLETE_2YRS,CNT_NONCOMPLETE_2YRS,COMPLETE_ALL_RATIO,COMPLETE_ALL_2YRS_RATIO,NONCOMPLETE_ALL_RATIO,NONCOMPLETE_ALL_2YRS_RATIO
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i16,i16,i32,i32,i32,i32,i32,i32,…,f64,f64,f64,f64,i16,i16,i32,i32,i32,i32,i32,i32,f64,f64,i16,i16,f64,f64,i16,i16,i32,i32,i32,i32,i32,i32,f64,f64,i16,i16,f64,f64,u32,f64,f64,f64,f64
376243,-9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-18.0,9.0,-13.0,11.0,-9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-17.5,8.5,-13.25,11.0,-1,4,0,0,0,0,0,0,…,-8.0,7.0,-4.5,12.0,-1,4,0,0,0,0,0,0,0.0,0.0,-8,7,-4.5,12.0,-1,4,0,0,0,0,0,0,0.0,0.0,-8,7,-4.5,12.0,1,0.5,0.5,0.5,0.5
394473,-11.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-27.0,16.0,-19.0,18.0,-11.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-27.0,16.0,-19.0,18.0,-1,3,0,0,0,0,0,0,…,-22.0,21.0,-11.5,24.0,-1,3,0,0,0,0,0,0,0.0,0.0,-22,21,-11.5,24.0,-1,3,0,0,0,0,0,0,0.0,0.0,-22,21,-11.5,24.0,1,0.5,0.5,0.5,0.5
356411,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-24.0,18.0,-15.0,18.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-24.0,18.0,-15.0,18.0,-6,0,0,0,0,0,0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1.0,null,null
255255,-46.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-53.0,7.0,-49.0,8.0,-49.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-58.0,7.5,-53.5,9.5,-2,9,0,0,0,0,0,0,…,-3.0,1.0,-2.5,10.0,-2,9,0,0,0,0,0,0,0.0,0.0,-3,1,-2.5,10.0,-2,9,0,0,0,0,0,0,0.0,0.0,-3,1,-2.5,10.0,1,0.5,null,0.5,1.0
182341,-31.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-39.0,8.0,-35.0,17.0,-16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-24.0,8.0,-20.0,11.0,-1,26,0,0,0,0,0,0,…,-11.0,10.0,-6.0,36.0,-1,26,0,0,0,0,0,0,0.0,0.0,-11,10,-6.0,36.0,-1,26,0,0,

To enable initial removal of multicollinear and unimportant features, missing values are simply filled with 0.

In [33]:
pos_cash_agg_filled = (
    pos_cash_agg.filter(c.SK_ID_CURR.is_in(idx_train))
    .fill_null(value=0)
    .fill_nan(value=0)
    .join(**appls_join)
    .to_pandas()
)
X_pos_cash = pos_cash_agg_filled.drop(columns=["TARGET", "SK_ID_CURR"])
y_pos_cash = pos_cash_agg_filled["TARGET"]

In [34]:
drop_corr_pos_cash = get_drop_multicollinear(
    X_pos_cash, y_pos_cash, "drop_corr_pos_cash", refit=False
)
X_pos_cash = X_pos_cash.drop(columns=drop_corr_pos_cash)

Number of quasi-constant and multicollinear aggregated features to drop: 266


In [35]:
drop_boruta_pos_cash = get_drop_by_boruta(
    X_pos_cash, y_pos_cash, "drop_boruta_pos_cash", refit=False
)
X_pos_cash = X_pos_cash.drop(columns=drop_boruta_pos_cash)
pos_cash_col_num = X_pos_cash.shape[1]
print(f"Number of selected features: {pos_cash_col_num}")

Number of unimportant aggregated features to drop using Boruta: 3
Number of selected features: 77


Finally, 77 aggregated features for POS and cash loan balance data are saved for later use. Note, that missing values are not imputed with 0 in the saved table.

In [36]:
pos_cash_agg.drop(drop_corr_pos_cash + drop_boruta_pos_cash).write_csv(
    "./data_agg/pos_cash_agg.csv"
)

In [37]:
del (
    pos_cash,
    pos_cash_agg,
    pos_cash_agg_initial,
    pos_cash_agg_filled,
    X_pos_cash,
    y_pos_cash,
)

### 3.5. <a id='toc3_5_'></a>Balance of Previous Credit Cards from Home Credit  [&#8593;](#toc0_)

First, various ratio features are created to diminish the impact of absolute values. To enable this, missing values are simply imputed with 0.

In [38]:
card = (
    pl.read_csv("./data/credit_card_balance.csv")
    .pipe(credit_data_utils.initial_pos_cash_credit_card_installments_cleaning)
    .pipe(credit_data_utils.reduce_memory_usage_pl)
)
card = card.fill_null(0).with_columns(
    BALANCE_LIMIT_RATIO=c.AMT_BALANCE / (c.AMT_CREDIT_LIMIT_ACTUAL + 1),
    DRAWINGS_ATM_RATIO=c.AMT_DRAWINGS_ATM_CURRENT
    / (c.AMT_DRAWINGS_CURRENT + 1),
    DRAWINGS_OTHER_RATIO=c.AMT_DRAWINGS_OTHER_CURRENT
    / (c.AMT_DRAWINGS_CURRENT + 1),
    DRAWINGS_POS_RATIO=c.AMT_DRAWINGS_POS_CURRENT
    / (c.AMT_DRAWINGS_CURRENT + 1),
    PAYMENT_INST_RATIO=c.AMT_PAYMENT_CURRENT / (c.AMT_INST_MIN_REGULARITY + 1),
    PAYMENT_TOTAL_INST_RATIO=c.AMT_PAYMENT_TOTAL_CURRENT
    / (c.AMT_INST_MIN_REGULARITY + 1),
    PRINCIPAL_TOTAL_RECEIVABLE_RATIO=c.AMT_RECEIVABLE_PRINCIPAL
    / (c.AMT_TOTAL_RECEIVABLE + 1),
)
card.head()

Size before cleaning: 707.31 MB
Size after cleaning: 707.31 MB
Number of columns left: 23
Size before memory reduction: 707.31 MB
Initial data types Counter({Int64: 11, Float64: 11, String: 1})
Size after memory reduction: 278.80 MB
Final data types Counter({Float32: 11, Int16: 7, Int32: 3, Int8: 1, Categorical(ordering='physical'): 1})


SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,BALANCE_LIMIT_RATIO,DRAWINGS_ATM_RATIO,DRAWINGS_OTHER_RATIO,DRAWINGS_POS_RATIO,PAYMENT_INST_RATIO,PAYMENT_TOTAL_INST_RATIO,PRINCIPAL_TOTAL_RECEIVABLE_RATIO
i32,i32,i16,f32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i16,i8,i16,i16,cat,i16,i16,f64,f64,f64,f64,f64,f64,f64
2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,1800.0,1800.0,0.0,0.0,0.0,0,1,0,1,35,"""Active""",0,0,0.000422,0.0,0.0,0.998862,1.057999,1.057999,0.0
2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.0,2250.0,2250.0,60175.078125,64875.554688,64875.554688,1,1,0,0,69,"""Active""",0,0,1.421647,0.999556,0.0,0.0,0.999556,0.999556,0.927532
1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.0,2250.0,2250.0,26926.425781,31460.085938,31460.085938,0,0,0,0,30,"""Active""",0,0,0.0707,0.0,0.0,0.0,0.999556,0.999556,0.855864
1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,11925.0,11925.0,224949.28125,233048.96875,233048.96875,1,1,0,0,10,"""Active""",0,0,1.051427,0.999556,0.0,0.0,1.010871,1.010871,0.965241
1891521,126868,-1,453919.46875,450000,0.0,11547.0,0.0,11547.0,22924.890625,27000.0,27000.0,443044.40625,453919.46875,453919.46875,0,1,0,1,101,"""Active""",0,0,1.008708,0.0,0.0,0.999913,1.177708,1.177708,0.97604


Initially, some aggregations are done on individual loans to have information on mean, median, sum, last, minimum or maximum values.

In [39]:
card_agg_initial = card.group_by(["SK_ID_PREV", "SK_ID_CURR"]).agg(
    cs.exclude(["SK_ID_PREV", "SK_ID_CURR"])
    .filter(c.MONTHS_BALANCE == c.MONTHS_BALANCE.max())
    .first()
    .name.suffix("_LAST"),
    cs.contains(["SK_DPD", "CNT_", "AMT_"]).max().name.suffix("_MAX"),
    cs.contains(["SK_DPD", "CNT_", "AMT_"]).sum().name.suffix("_SUM"),
    cs.contains(["SK_DPD", "CNT_", "AMT_"]).mean().round().name.suffix("_AVG"),
    cs.contains(["SK_DPD", "CNT_", "AMT_"]).median().name.suffix("_MEDIAN"),
    cs.contains("_RATIO").max().name.suffix("_MAX"),
    cs.contains("_RATIO").min().name.suffix("_MIN"),
    cs.contains("_RATIO").mean().name.suffix("_AVG"),
    cs.contains("_RATIO").median().name.suffix("_MEDIAN"),
    MONTHS_BALANCE_FIRST=c.MONTHS_BALANCE.min(),
    MONTHS_BALANCE_DURATION=c.MONTHS_BALANCE.max() - c.MONTHS_BALANCE.min(),
    MONTHS_BALANCE_MEDIAN=c.MONTHS_BALANCE.median(),
)
card_agg_initial

SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE_LAST,AMT_BALANCE_LAST,AMT_CREDIT_LIMIT_ACTUAL_LAST,AMT_DRAWINGS_ATM_CURRENT_LAST,AMT_DRAWINGS_CURRENT_LAST,AMT_DRAWINGS_OTHER_CURRENT_LAST,AMT_DRAWINGS_POS_CURRENT_LAST,AMT_INST_MIN_REGULARITY_LAST,AMT_PAYMENT_CURRENT_LAST,AMT_PAYMENT_TOTAL_CURRENT_LAST,AMT_RECEIVABLE_PRINCIPAL_LAST,AMT_RECIVABLE_LAST,AMT_TOTAL_RECEIVABLE_LAST,CNT_DRAWINGS_ATM_CURRENT_LAST,CNT_DRAWINGS_CURRENT_LAST,CNT_DRAWINGS_OTHER_CURRENT_LAST,CNT_DRAWINGS_POS_CURRENT_LAST,CNT_INSTALMENT_MATURE_CUM_LAST,NAME_CONTRACT_STATUS_LAST,SK_DPD_LAST,SK_DPD_DEF_LAST,BALANCE_LIMIT_RATIO_LAST,DRAWINGS_ATM_RATIO_LAST,DRAWINGS_OTHER_RATIO_LAST,DRAWINGS_POS_RATIO_LAST,PAYMENT_INST_RATIO_LAST,PAYMENT_TOTAL_INST_RATIO_LAST,PRINCIPAL_TOTAL_RECEIVABLE_RATIO_LAST,AMT_BALANCE_MAX,AMT_CREDIT_LIMIT_ACTUAL_MAX,AMT_DRAWINGS_ATM_CURRENT_MAX,AMT_DRAWINGS_CURRENT_MAX,AMT_DRAWINGS_OTHER_CURRENT_MAX,AMT_DRAWINGS_POS_CURRENT_MAX,AMT_INST_MIN_REGULARITY_MAX,…,CNT_DRAWINGS_CURRENT_MEDIAN,CNT_DRAWINGS_OTHER_CURRENT_MEDIAN,CNT_DRAWINGS_POS_CURRENT_MEDIAN,CNT_INSTALMENT_MATURE_CUM_MEDIAN,SK_DPD_MEDIAN,SK_DPD_DEF_MEDIAN,BALANCE_LIMIT_RATIO_MAX,DRAWINGS_ATM_RATIO_MAX,DRAWINGS_OTHER_RATIO_MAX,DRAWINGS_POS_RATIO_MAX,PAYMENT_INST_RATIO_MAX,PAYMENT_TOTAL_INST_RATIO_MAX,PRINCIPAL_TOTAL_RECEIVABLE_RATIO_MAX,BALANCE_LIMIT_RATIO_MIN,DRAWINGS_ATM_RATIO_MIN,DRAWINGS_OTHER_RATIO_MIN,DRAWINGS_POS_RATIO_MIN,PAYMENT_INST_RATIO_MIN,PAYMENT_TOTAL_INST_RATIO_MIN,PRINCIPAL_TOTAL_RECEIVABLE_RATIO_MIN,BALANCE_LIMIT_RATIO_AVG,DRAWINGS_ATM_RATIO_AVG,DRAWINGS_OTHER_RATIO_AVG,DRAWINGS_POS_RATIO_AVG,PAYMENT_INST_RATIO_AVG,PAYMENT_TOTAL_INST_RATIO_AVG,PRINCIPAL_TOTAL_RECEIVABLE_RATIO_AVG,BALANCE_LIMIT_RATIO_MEDIAN,DRAWINGS_ATM_RATIO_MEDIAN,DRAWINGS_OTHER_RATIO_MEDIAN,DRAWINGS_POS_RATIO_MEDIAN,PAYMENT_INST_RATIO_MEDIAN,PAYMENT_TOTAL_INST_RATIO_MEDIAN,PRINCIPAL_TOTAL_RECEIVABLE_RATIO_MEDIAN,MONTHS_BALANCE_FIRST,MONTHS_BALANCE_DURATION,MONTHS_BALANCE_MEDIAN
i32,i32,i16,f32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i16,i8,i16,i16,cat,i16,i16,f64,f64,f64,f64,f64,f64,f64,f32,i32,f32,f32,f32,f32,f32,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i16,i16,f64
2037377,289902,-1,131294.34375,135000,0.0,0.0,0.0,0.0,6774.660156,6975.0,6975.0,128518.023438,131294.34375,131294.34375,0,0,0,0,11,"""Active""",0,0,0.972543,0.0,0.0,0.0,1.02942,1.02942,0.978847,151977.828125,135000,45900.0,45900.0,0.0,6255.0,7942.274902,…,0.0,0.0,0.0,5.5,0.0,0.0,1.125753,0.999978,0.0,0.139136,20.432736,20.432736,0.989527,0.054965,0.0,0.0,0.0,0.0,0.0,0.0,0.962212,0.321731,0.0,0.011595,2.602562,2.602562,0.869942,1.032899,0.0,0.0,0.0,1.034409,1.034409,0.95133,-12,11,-6.5
1189095,231608,-1,0.0,450000,0.0,156.960007,0.0,0.0,2250.0,7757.189941,7757.189941,0.0,0.0,0.0,0,0,0,0,25,"""Completed""",0,0,0.0,0.0,0.0,0.0,3.446108,3.446108,0.0,453250.03125,450000,405000.0,405000.0,0.0,22784.400391,23289.300781,…,1.0,0.0,0.0,9.5,0.0,0.0,1.00722,0.999998,0.0,0.999926,4839.390137,4839.390137,0.999991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.590352,0.557397,0.0,0.036317,346.854922,346.854922,0.640328,0.909831,0.961957,0.0,0.0,1.207318,1.207318,0.974022,-32,31,-16.5
2392920,132481,-1,85792.09375,135000,45000.0,63337.816406,0.0,18337.814453,2250.0,23181.570312,18521.550781,85086.945312,85792.09375,85792.09375,1,9,0,8,5,"""Active""",0,0,0.635492,0.710465,0.0,0.289519,10.298343,8.228143,0.991769,85792.09375,135000,45000.0,63337.816406,0.0,38730.105469,2250.0,…,7.0,0.0,7.0,2.0,0.0,0.0,0.635492,0.710465,0.0,0.999974,15750.0,13500.0,0.999964,0.0232,0.0,0.0,0.289519,0.0,0.0,0.991769,0.222439,0.101495,0.0,0.898432,2257.794493,1935.499004,0.997884,0.204462,0.0,0.0,0.99996,10.345942,9.346386,0.999681,-7,6,-4.0
1927949,323730,-1,148956.34375,157500,0.0,0.0,0.0,0.0,7875.0,7875.0,7875.0,143028.3125,148956.34375,148956.34375,0,0,0,0,72,"""Active""",0,0,0.945749,0.0,0.0,0.0,0.999873,0.999873,0.960196,169195.640625,157500,157500.0,157500.0,0.0,0.0,7875.0,…,0.0,0.0,0.0,37.

Again, function is written to apply aggregations over different groups of data.

In [40]:
def card_aggregations_func(
    df: pl.DataFrame, suffix: str = "", left_df: pl.DataFrame = None
) -> pl.DataFrame:
    """Function to aggregate credit card balance data and optionally add
    suffix to new columns or join with other dataframe."""
    agg_df = df.group_by("SK_ID_CURR").agg(
        cs.contains(["MONTHS_", "CNT_", "SK_DPD_"])
        .mean()
        .round()
        .name.suffix("_AVG"),
        cs.contains(["AMT_", "_RATIO"]).mean().name.suffix("_AVG"),
        (cs.all() - cs.contains(["SK_ID_", "NAME_"]))
        .median()
        .name.suffix("_MEDIAN"),
        (cs.all() - cs.contains(["SK_ID_", "NAME_"]))
        .max()
        .name.suffix("_MAX"),
        (cs.all() - cs.contains(["SK_ID_", "NAME_"]))
        .min()
        .name.suffix("_MIN"),
        CNT=c.SK_ID_CURR.count(),
    )
    agg_df = agg_df.rename(
        lambda col: col + suffix if col != "SK_ID_CURR" else col
    )
    if left_df is not None:
        agg_df = left_df.join(other=agg_df, on="SK_ID_CURR", how="left")
    return agg_df

These aggregations are applied over the whole dataset and also only on the information from the last two years, last two months.

In [41]:
card_agg = card_aggregations_func(card_agg_initial, "_ALL")
card_agg = card_agg_initial.filter(
    c.NAME_CONTRACT_STATUS_LAST == "Active"
).pipe(card_aggregations_func, "ACTIVE", card_agg)
card_agg = card.filter(c.MONTHS_BALANCE >= -24).pipe(
    card_aggregations_func, "_2YRS", card_agg
)
card_agg = card.filter(c.MONTHS_BALANCE >= -2).pipe(
    card_aggregations_func, "_2MTHS", card_agg
)

To enable initial removal of multicollinear and unimportant features, missing values are simply filled with 0.

In [42]:
card_agg_filled = (
    card_agg.filter(c.SK_ID_CURR.is_in(idx_train))
    .fill_null(value=0)
    .fill_nan(value=0)
    .join(**appls_join)
    .to_pandas()
)
X_card = card_agg_filled.drop(columns=["TARGET", "SK_ID_CURR"])
y_card = card_agg_filled["TARGET"]

In [43]:
drop_corr_card = get_drop_multicollinear(
    X_card, y_card, "drop_corr_card", refit=False
)
X_card = X_card.drop(columns=drop_corr_card)

Number of quasi-constant and multicollinear aggregated features to drop: 1159


In [44]:
drop_boruta_card = get_drop_by_boruta(
    X_card, y_card, "drop_boruta_card", refit=False
)
X_card = X_card.drop(columns=drop_boruta_card)
card_col_num = X_card.shape[1]
print(f"Number of selected features: {card_col_num}")

Number of unimportant aggregated features to drop using Boruta: 50
Number of selected features: 81


Finally, 81 aggregated feature for credit card balance data is saved for later use. Note, that missing values are not imputed with 0 in the saved table.

In [45]:
card_agg.drop(drop_corr_card + drop_boruta_card).write_csv(
    "./data_agg/card_agg.csv"
)

In [46]:
del (
    card,
    card_agg,
    card_agg_initial,
    card_agg_filled,
    X_card,
    y_card,
)

### 3.6. <a id='toc3_6_'></a>Installments on Previous Loans from Home Credit  [&#8593;](#toc0_)

A few missing values are imputed with 0 to enable feature engineering. First some relative features are created.

In [47]:
inst = (
    pl.read_csv("./data/installments_payments.csv")
    .pipe(credit_data_utils.initial_pos_cash_credit_card_installments_cleaning)
    .pipe(credit_data_utils.reduce_memory_usage_pl)
)
inst = inst.fill_null(0).with_columns(
    DAYS_LATE=c.DAYS_INSTALMENT - c.DAYS_ENTRY_PAYMENT,
    PAYMENT_INSTALMENT_DIFF=c.AMT_PAYMENT - c.AMT_INSTALMENT,
    PAYMENT_INSTALMENT_RATIO=c.AMT_PAYMENT / (c.AMT_INSTALMENT + 1),
)
inst.head()

Size before cleaning: 832.03 MB
Size after cleaning: 832.03 MB
Number of columns left: 8
Size before memory reduction: 832.03 MB
Initial data types Counter({Int64: 6, Float64: 2})
Size after memory reduction: 338.98 MB
Final data types Counter({Int32: 3, Int16: 3, Float32: 2})


SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_LATE,PAYMENT_INSTALMENT_DIFF,PAYMENT_INSTALMENT_RATIO
i32,i32,i16,i16,i16,i32,f32,f32,i32,f32,f64
1054186,161674,1,6,-1180,-1187,6948.359863,6948.359863,7,0.0,0.999856
1330831,151639,0,34,-2156,-2156,1716.525024,1716.525024,0,0.0,0.999418
2085231,193053,2,1,-63,-63,25425.0,25425.0,0,0.0,0.999961
2452527,199697,1,3,-2418,-2426,24350.130859,24350.130859,8,0.0,0.999959
2714724,167756,1,2,-1383,-1366,2165.040039,2160.584961,-17,-4.455078,0.997482


Initially, some aggregations are done on installments for individual loans to have information on mean, median, sum, last, minimum or maximum values.

In [48]:
inst_agg_initial = inst.group_by(["SK_ID_PREV", "SK_ID_CURR"]).agg(
    cs.exclude(["SK_ID_PREV", "SK_ID_CURR"])
    .filter(c.DAYS_INSTALMENT == c.DAYS_INSTALMENT.max())
    .first()
    .name.suffix("_LAST"),
    cs.contains(["DAYS_"])
    .filter(c.DAYS_INSTALMENT == c.DAYS_INSTALMENT.min())
    .first()
    .name.suffix("_FIRST"),
    cs.contains(["AMT_", "DAYS_LATE", "_DIFF", "_RATIO"])
    .max()
    .name.suffix("_MAX"),
    cs.contains(["AMT_", "DAYS_LATE", "_DIFF", "_RATIO"])
    .min()
    .name.suffix("_MIN"),
    cs.contains(["AMT_", "DAYS_LATE", "_DIFF", "_RATIO"])
    .median()
    .name.suffix("_MEDIAN"),
    cs.contains(["AMT_", "_DIFF", "_RATIO"]).mean().name.suffix("_AVG"),
    cs.contains(["AMT_", "DAYS_LATE", "_DIFF"]).sum().name.suffix("_SUM"),
    DAYS_LATE_AVG=c.DAYS_LATE.mean().round(),
    DAYS_INSTALMENT_DURATION=c.DAYS_INSTALMENT.max() - c.DAYS_INSTALMENT.min(),
    DAYS_ENTRY_PAYMENT_DURATION=c.DAYS_ENTRY_PAYMENT.max()
    - c.DAYS_ENTRY_PAYMENT.min(),
    DAYS_INSTALMENT_MEDIAN=c.DAYS_INSTALMENT.median(),
    DAYS_ENTRY_PAYMENT_MEDIAN=c.DAYS_ENTRY_PAYMENT.median(),
)
inst_agg_initial.head()

SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION_LAST,NUM_INSTALMENT_NUMBER_LAST,DAYS_INSTALMENT_LAST,DAYS_ENTRY_PAYMENT_LAST,AMT_INSTALMENT_LAST,AMT_PAYMENT_LAST,DAYS_LATE_LAST,PAYMENT_INSTALMENT_DIFF_LAST,PAYMENT_INSTALMENT_RATIO_LAST,DAYS_INSTALMENT_FIRST,DAYS_ENTRY_PAYMENT_FIRST,DAYS_LATE_FIRST,AMT_INSTALMENT_MAX,AMT_PAYMENT_MAX,DAYS_LATE_MAX,PAYMENT_INSTALMENT_DIFF_MAX,PAYMENT_INSTALMENT_RATIO_MAX,AMT_INSTALMENT_MIN,AMT_PAYMENT_MIN,DAYS_LATE_MIN,PAYMENT_INSTALMENT_DIFF_MIN,PAYMENT_INSTALMENT_RATIO_MIN,AMT_INSTALMENT_MEDIAN,AMT_PAYMENT_MEDIAN,DAYS_LATE_MEDIAN,PAYMENT_INSTALMENT_DIFF_MEDIAN,PAYMENT_INSTALMENT_RATIO_MEDIAN,AMT_INSTALMENT_AVG,AMT_PAYMENT_AVG,PAYMENT_INSTALMENT_DIFF_AVG,PAYMENT_INSTALMENT_RATIO_AVG,AMT_INSTALMENT_SUM,AMT_PAYMENT_SUM,DAYS_LATE_SUM,PAYMENT_INSTALMENT_DIFF_SUM,DAYS_LATE_AVG,DAYS_INSTALMENT_DURATION,DAYS_ENTRY_PAYMENT_DURATION,DAYS_INSTALMENT_MEDIAN,DAYS_ENTRY_PAYMENT_MEDIAN
i32,i32,i16,i16,i16,i32,f32,f32,i32,f32,f64,i16,i32,i32,f32,f32,i32,f32,f64,f32,f32,i32,f32,f64,f32,f32,f64,f32,f64,f32,f32,f32,f64,f32,f32,i32,f32,f64,i16,i32,f64,f64
1137312,164489,2,15,-1294,-1315,57861.179688,57861.179688,21,0.0,0.999983,-1714,-1744,30,57861.179688,57861.179688,33,0.0,0.999983,5970.375,5970.375,9,0.0,0.999833,5970.375,5970.375,21.0,0.0,0.999833,9429.762695,9429.762695,0.0,0.999843,141446.4375,141446.4375,325,0.0,22.0,420,429,-1504.0,-1534.0
1053286,170192,2,23,-20,-20,13536.450195,13536.450195,0,0.0,0.999926,-680,-699,19,13536.450195,13536.450195,34,0.0,0.999926,6882.839844,77.400002,-6,-6805.439941,0.011244,6882.839844,6882.839844,2.5,0.0,0.999855,7120.47168,5891.390625,-1229.078491,0.821312,199373.203125,164958.9375,185,-34414.199219,7.0,660,679,-335.0,-335.5
2702769,102122,2,8,-71,-72,14682.959961,14682.959961,1,0.0,0.999932,-281,-286,5,14682.959961,14682.959961,30,0.0,0.999932,5178.194824,5178.194824,0,0.0,0.999807,5178.194824,5178.194824,4.0,0.0,0.999807,6366.290527,6366.290527,0.0,0.999823,50930.324219,50930.324219,96,0.0,12.0,210,214,-176.0,-191.0
1840786,175114,1,12,-446,-453,21039.570312,21039.570312,7,0.0,0.999952,-776,-778,2,21179.699219,21179.699219,38,0.0,0.999953,21039.570312,1.8,-3,-21177.898438,0.000085,21179.699219,21174.300781,2.0,-0.899414,0.99991,21170.941406,15876.018555,-5294.924805,0.749965,338735.0625,254016.296875,139,-84718.796875,9.0,330,325,-611.0,-609.0
2671380,111601,1,10,-450,-457,8102.024902,8102.024902,7,0.0,0.999877,-720,-736,16,8109.765137,8109.765137,18,0.0,0.999877,8102.024902,8102.024902,7,0.0,0.999877,8109.765137,8109.765137,12.0,0.0,0.999877,8108.991211,8108.991211,0.0,0.999877,81089.914062,81089.914062,120,0.0,12.0,270,279,-585.0,-596.0


Again, function is written to apply aggregations over different groups of data.

In [49]:
def inst_aggregations_func(
    df: pl.DataFrame, suffix: str = "", left_df: pl.DataFrame = None
) -> pl.DataFrame:
    """Function to aggregate installments payments data and optionally add
    suffix to new columns or join with other dataframe."""
    agg_df = df.group_by("SK_ID_CURR").agg(
        cs.contains(["DAYS_"]).mean().round().name.suffix("_AVG"),
        cs.contains(["AMT_", "_DIFF", "_RATIO"]).mean().name.suffix("_AVG"),
        cs.contains(["AMT_", "_DIFF", "_RATIO", "DAYS_"])
        .median()
        .name.suffix("_MEDIAN"),
        cs.contains(["AMT_", "_DIFF", "_RATIO", "DAYS_"])
        .max()
        .name.suffix("_MAX"),
        cs.contains(["AMT_", "_DIFF", "_RATIO", "DAYS_"])
        .min()
        .name.suffix("_MIN"),
        cs.contains(["AMT_", "_DIFF", "DAYS_LATE"]).sum().name.suffix("_SUM"),
        CNT=c.SK_ID_CURR.count(),
    )
    agg_df = agg_df.rename(
        lambda col: col + suffix if col != "SK_ID_CURR" else col
    )
    if left_df is not None:
        agg_df = left_df.join(other=agg_df, on="SK_ID_CURR", how="left")
    return agg_df

These aggregations are applied over the whole dataset, only credit card installments, only non-credit card installments, only on the installments from the last one and two years.

In [50]:
inst_agg = inst_aggregations_func(inst_agg_initial, "_ALL")
inst_agg = inst_agg_initial.filter(c.NUM_INSTALMENT_VERSION_LAST == 0).pipe(
    inst_aggregations_func, "_CARD", inst_agg
)
inst_agg = inst_agg_initial.filter(c.NUM_INSTALMENT_VERSION_LAST != 0).pipe(
    inst_aggregations_func, "_NONCARD", inst_agg
)
inst_agg = inst.filter(c.DAYS_ENTRY_PAYMENT >= -365).pipe(
    inst_aggregations_func, "_1YR", inst_agg
)
inst_agg = inst.filter(c.DAYS_ENTRY_PAYMENT >= (-365 * 2)).pipe(
    inst_aggregations_func, "_2YRS", inst_agg
)

To enable initial removal of multicollinear and unimportant features, missing values are simply filled with 0.

In [51]:
inst_agg_filled = (
    inst_agg.filter(c.SK_ID_CURR.is_in(idx_train))
    .fill_null(value=0)
    .fill_nan(value=0)
    .join(**appls_join)
    .to_pandas()
)
X_inst = inst_agg_filled.drop(columns=["TARGET", "SK_ID_CURR"])
y_inst = inst_agg_filled["TARGET"]

In [52]:
drop_corr_inst = get_drop_multicollinear(
    X_inst, y_inst, "drop_corr_inst", refit=False
)
X_inst = X_inst.drop(columns=drop_corr_inst)

Number of quasi-constant and multicollinear aggregated features to drop: 410


In [53]:
drop_boruta_inst = get_drop_by_boruta(
    X_inst, y_inst, "drop_boruta_inst", refit=False
)
X_inst = X_inst.drop(columns=drop_boruta_inst)
inst_col_num = X_inst.shape[1]
print(f"Number of selected features: {inst_col_num}")

Number of unimportant aggregated features to drop using Boruta: 26
Number of selected features: 164


Finally, 164 aggregated features for installment data are saved for later use. Note, that missing values are not imputed with 0 in the saved table.

In [54]:
inst_agg.drop(drop_corr_inst + drop_boruta_inst).write_csv(
    "./data_agg/inst_agg.csv"
)

In [55]:
del (
    inst,
    inst_agg,
    inst_agg_initial,
    inst_agg_filled,
    X_inst,
    y_inst,
)

### 3.7. <a id='toc3_7_'></a>Summary on Feature Engineering [&#8593;](#toc0_)

1. A total of 38 features (original and engineered) from the main home credit application table will be further used in modeling. Among them, the mean of the target value of the 500 closest neighbors based on external sources and the credit-annuity ratio will be included.
2. A total of 109 engineered features from the tables on previous credits and their balances from other institutions will be further used in modeling. Aggregations here were applied not only to the whole table but also separately to active and closed credits, as well as to card and consumer credits.
3. A total of 271 engineered features describing the previous applications for Home Credit loans will be further used in modeling. Aggregations here were applied not only to the whole table but also separately for applications of different statuses and types.
4. A total of 77 engineered features describing the balance of previous point-of-sales and cash loans from Home Credit will be further used in modeling. Aggregations were applied to the whole table, to active loans only, and to the last two years.
5. A total of 81 engineered features describing the balance of previous credit cards from Home Credit will be further used in modeling. Aggregations were applied to all credit cards, only to active credit cards, and only to the last two years and the last two months.
6. A total of 164 engineered features describing installments on previous loans from Home Credit will be further used in modeling. Aggregations were applied to the whole table, only to credit card installments, non-credit card installments, and to the installments from the last one year and the last two years.